In [1]:
pip install nltk

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to C:\Users\VISHAL/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VISHAL/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
import nltk
nltk.download('omw')

[nltk_data] Downloading package omw to C:\Users\VISHAL/nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

In [4]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [7]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('/content/drive/MyDrive/AI_CHATBOT/intents.json').read() #Copy the path of your json file which is uploaded on your google drive
intents = json.loads(data_file)

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

In [10]:
print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

320 documents
116 classes ['Appreciation', 'BMI', 'Concern', 'Criticism', 'O2_level', 'Personal', 'Please', 'WBC_count', 'acidity', 'acne_causes', 'age_skin', 'ava_sp', 'ayu_asthma', 'ayu_cold', 'ayu_digestion', 'ayu_hair_loss', 'ayu_heart', 'ayu_hormonal_imbalance', 'ayu_immune', 'ayu_joint_pain', 'ayu_skin', 'ayu_stress', 'balanced_meal', 'bf_ideas', 'blood_thinners', 'book_sp', 'causes_hair_loss', 'chest_pain', 'cold', 'cough', 'country_sp', 'creation_sp', 'cut_sugar', 'dark_spots', 'diet', 'diet_skin', 'evil_sp', 'exercise', 'fail_sp', 'fasting', 'fever', 'fiber', 'gandhi_sp', 'giddiness', 'goodbye', 'green_tea', 'greeting', 'hair_care_routine', 'hair_growth', 'headache', 'healthy_fats', 'high_blood_pressure', 'home_allergy', 'home_burns', 'home_cold', 'home_headache', 'home_indigestion', 'home_joint_pain', 'home_sleep', 'home_stress', 'home_throat', 'human_sp', 'hurt_sp', 'hydration', 'inflammation', 'leg_pain', 'lifestyle', 'love_sp', 'maintain_healthy_hair', 'mark_sp', 'meal_pla

In [11]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [12]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-12-1a891a7a8859>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [13]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [14]:
from tensorflow.keras.optimizers import SGD

sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)


Epoch 1/200
64/64 [==============================] - 1s 3ms/step - loss: 4.7559 - accuracy: 0.0375
Epoch 2/200
64/64 [==============================] - 0s 3ms/step - loss: 4.7009 - accuracy: 0.0562
Epoch 3/200
64/64 [==============================] - 0s 3ms/step - loss: 4.6358 - accuracy: 0.0594
Epoch 4/200
64/64 [==============================] - 0s 3ms/step - loss: 4.5672 - accuracy: 0.0562
Epoch 5/200
64/64 [==============================] - 0s 2ms/step - loss: 4.4753 - accuracy: 0.0656
Epoch 6/200
64/64 [==============================] - 0s 3ms/step - loss: 4.3588 - accuracy: 0.0938
Epoch 7/200
64/64 [==============================] - 0s 3ms/step - loss: 4.2393 - accuracy: 0.0938
Epoch 8/200
64/64 [==============================] - 0s 3ms/step - loss: 4.1014 - accuracy: 0.1125
Epoch 9/200
64/64 [==============================] - 0s 3ms/step - loss: 4.0686 - accuracy: 0.1031
Epoch 10/200
64/64 [==============================] - 0s 2ms/step - loss: 3.9056 - accuracy: 0.1125
Epoch 11/